In [ ]:
#Librerias a utilizar
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler , LabelEncoder
import numpy as np
from sklearn.neighbors import NearestNeighbors
# Para evaluar el modelo
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

In [ ]:
#Cargar DataFrame
df = pd.read_csv('customer.csv')
#Verificar Caracteristicas totales del DataFRame
print(f'Caracteristicas Total del Dataframe: Filas: {df.shape[0]} , Columnas:{df.shape[1]}')
#Veriricar los valores Nulos
datos_nulos = [i for i in df.isna().sum() if i>1]
print(f'Datos Nulos: {datos_nulos[0]}')
porc_nulos = datos_nulos[0] / df.shape[0]
print(f'Porcentaje de Valores nulos en la Data: {round(porc_nulos*100,1)} %')
df.columns
#Eliminar Datos nulos
df.dropna(inplace=True)
print(f'Dataframe Sin Valores Nulos: Filas: {df.shape[0]} , Columnas:{df.shape[1]}')

In [ ]:
#Funciones para Visualizacion de Outliers y Tratamiento de los mismos

def crearBoxPlot_Numericos(dataframe):
    """
    Se realizara el diagrama de caja y bigotes de las columnas
    numericas que no contengan 'palabra a eliminar'previemente descritas en el nombre.
    Retornara el grafico que nos muestra los
    datos outliers.

    """
    col_grafico = []
    palabra_a_eliminar=['ID','Year','Kid','Teen','Dt','Recency','Num','Accepted','Z_','Response','Complain']
    columnas_a_eliminar = [col for col in dataframe.columns if any(p in col for p in palabra_a_eliminar)]
    df_outlier = dataframe.drop(columnas_a_eliminar,axis=1)
    for col in df_outlier.columns:
        if pd.api.types.is_numeric_dtype(df_outlier[col]):
            col_grafico.append(col)
            sns.boxplot(df_outlier[col])
            plt.title(f'Boxplot de {col}')
            plt.show()
    return col_grafico
                
#Funcion para tratar los outliers
def manejar_outliers(dataframe, columnas):
    """
    Identifica y elimina outliers en múltiples columnas de un DataFrame.
    Args:
        dataframe: El DataFrame de Pandas.
        columnas: Una lista con los nombres de las columnas a procesar.
    Returns:
        Un nuevo DataFrame sin los outliers en las columnas especificadas.
    """
    for columna in columnas:
        if columna in dataframe.columns:
            Q1 = dataframe[columna].quantile(0.25)
            Q3 = dataframe[columna].quantile(0.75)
            IQR = Q3 - Q1
            # Crear una máscara booleana para filtrar los outliers
            mascara_outliers = (dataframe[columna] < (Q1 - 1.5 * IQR)) | (dataframe[columna] > (Q3 + 1.5 * IQR))
            # Reemplazar outliers con la mediana de la columna
            mediana = dataframe[columna].median()
            dataframe.loc[mascara_outliers, columna] = mediana
            
        # Verificar si se han aplicado los cambios en cada columna
            print(f'Procesando columna: {columna}, Mediana: {mediana}, Outliers reemplazados: {mascara_outliers.sum()}')
        else:
            print(f"Columna {columna} no encontrada en el DataFrame")
    return dataframe
#Visualizar outliers y recoger la lista de columnas a trabajar para los outliers
columnas_atrabajar = crearBoxPlot_Numericos(df)
print(columnas_atrabajar)

In [ ]:
#Manejar los Outlier con la mediana de cada columna
df_sin_outliers = manejar_outliers(df,columnas_atrabajar)

In [ ]:
#Creando boxplot General para verificar que este libre de Outliers
sns.boxplot(df_sin_outliers[columnas_atrabajar])
# Etiquetas y título
plt.title('Boxplots de verificacion de Outliers')
plt.xlabel('Columnas')
plt.xticks(rotation=45)
plt.ylabel('Valores')
plt.show()

In [ ]:
#Listar dataFrame con sus detalles
df_sin_outliers.info()

In [ ]:
#Previsalizar los datos
df_sin_outliers.head()

In [ ]:
#Categoricas a nuemricas
# Iterar sobre las columnas categóricas
for column in df_sin_outliers.select_dtypes(include='object').columns:
    le = LabelEncoder()
    
    # Ajustar y transformar los valores de la columna categórica
    df_sin_outliers[column] = le.fit_transform(df_sin_outliers[column])

## Preparar Data para el Kmeans Standarizar

In [ ]:
# Estándarizar las variables

# Crear una instancia del algoritmo de estandarización
scaler = StandardScaler()

# Transformar (escalar) los datos
var_scaled = scaler.fit_transform(df_sin_outliers)

# Convertir el resultado a un DataFrame
df_scaled = pd.DataFrame(var_scaled, columns=df_sin_outliers.columns)

#Listar variables del dataset con sus detalles
df_scaled.head()

In [ ]:
# Trabajar solo con tres variables

df_to_cluster = df_scaled[['Year_Birth', 'Education', 'Income']]
df_to_cluster.head()

In [ ]:
# Crear y ajustar tres clústers
kmeans_model = KMeans(n_clusters=3)
kmeans_model.fit(df_to_cluster)

In [ ]:
# Asignar el clúster a cada fila del Dataset
df_to_cluster['cluster'] = kmeans_model.labels_
df_to_cluster.head()

In [ ]:
# Plotear

# Scatter
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(df_to_cluster['Year_Birth'], df_to_cluster['Education'], df_to_cluster['Income'],
                    c=df_to_cluster['cluster'],
                    cmap='viridis',
                    marker='o')

# Centroides
centers = kmeans_model.cluster_centers_

ax.scatter(centers[:, 0], centers[:, 1], centers[:, 2],
        s=300, c='red', marker='X', label='Centros')

ax.set_xlabel('Year_Birth')
ax.set_ylabel('Education')
ax.set_zlabel('Income')
plt.legend()
plt.colorbar(scatter, label='Cluster')

# Rotar el gráfico
ax.view_init(elev=20, azim=30)

plt.show()

In [ ]:
# Caracterizar Los Clúster

# Agregar al dataset original (sin escalar) la columna con el clúster estimado por el algoritmo
df_sin_outliers['clúster'] =  kmeans_model.labels_

# Separar cada clúster a su propio dataframe
cluster1 = df_sin_outliers[df_sin_outliers['clúster']==0]
cluster2 = df_sin_outliers[df_sin_outliers['clúster']==1]
cluster3 = df_sin_outliers[df_sin_outliers['clúster']==2]

In [ ]:
cluster1[['Year_Birth','Education', 'Income']].describe()
#En este cluster se puede observar que ahy personas entre 51(2024-1973)  y  29(2024-1995) años con ingresos q varian de 50447 a 105471

In [ ]:
cluster2[['Year_Birth','Education', 'Income']].describe()
#En este cluster se puede observar que ahy personas entre 131(2024-1893)  y  41(2024-1983) años con ingresos q varian de 5648 a 1113734

In [ ]:
cluster3[['Year_Birth','Education', 'Income']].describe()
#En este cluster se puede observar que ahy personas entre 74(2024-1950)  y  28(2024-1996) años con ingresos q varian de 1730 a 52413

In [ ]:
# Individuos por clúster

# Agregar al dataset original (sin escalar) la columna con el clúster estimado por el algoritmo
df_sin_outliers['cluster'] =  kmeans_model.labels_
# Recuento
df.groupby('cluster')['cluster'].count().reset_index(name='count')

## Metricas de evaluacion del modelo

In [ ]:
# Evaluation (Silhouette Coefficient)
sil_score = silhouette_score(df_to_cluster, kmeans_model.labels_)
print('Coeficiente de Silueta: {}'.format(sil_score))
# Evaluation (Inertia)
inertia_score = kmeans_model.inertia_
print('Inercia: {}'.format(inertia_score))
# Evaluation (Davies-Bouldin Score
db_score = davies_bouldin_score(df_to_cluster, kmeans_model.labels_)
print('Davies-Bouldin Score: {}'.format(db_score))

## Ejemplo evaluación recursiva de K-Means (varios K)

In [ ]:
# Número de clusters para probar
k_range = range(2, 11)

# Cálcular las métricas para todos los K
sil_scores = []
inertia_scores = []
db_scores = []

for k in k_range:
    # Entrenamiento del modelo para cada K
    kmeans_train = KMeans(n_clusters=k)
    labels = kmeans_train.fit_predict(df_to_cluster)

    # Evaluación Método: Silhouette Score
    sil_score = silhouette_score(df_to_cluster, labels)
    sil_scores.append(sil_score)

    #  Evaluación Método: Inertia
    inertia_score = kmeans_train.inertia_
    inertia_scores.append(inertia_score)

    #  Evaluación Método: Davies-Bouldin Score
    db_score = davies_bouldin_score(df_to_cluster, labels)
    db_scores.append(db_score)

In [ ]:
# Crear la figura
fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)

# Plot Silhouette Scores
axs[0].plot(k_range, sil_scores, marker='o', color='blue')
axs[0].set_title('Silhouette Score')

# Plot Inertia Scores
axs[1].plot(k_range, inertia_scores, marker='o', color='green')
axs[1].set_title('Inertia Score')

# Plot Davies-Bouldin Scores
axs[2].plot(k_range, db_scores, marker='o', color='red')
axs[2].set_title('Davies Bouldin Score')
axs[2].set_xlabel('Número de Clusters (k)')

plt.show()

### Kmeans con las variables dadas por las metricas

In [ ]:
# Crear y ajustar cuatro clústers
kmeans_model_2 = KMeans(n_clusters=4)
kmeans_model_2.fit(df_to_cluster)

In [ ]:
# Evaluation (Silhouette Coefficient)
sil_score_2 = silhouette_score(df_to_cluster, kmeans_model_2.labels_)
print('Coeficiente de Silueta: {}'.format(sil_score_2))
# Evaluation (Inertia)
inertia_score_2 = kmeans_model_2.inertia_
print('Inercia: {}'.format(inertia_score_2))
# Evaluation (Davies-Bouldin Score
db_score_2 = davies_bouldin_score(df_to_cluster, kmeans_model_2.labels_)
print('Davies-Bouldin Score: {}'.format(db_score_2))

In [ ]:
# Crear y ajustar diez clústers
kmeans_model_3 = KMeans(n_clusters=10)
kmeans_model_3.fit(df_to_cluster)

In [ ]:
# Evaluation (Silhouette Coefficient)
sil_score_3 = silhouette_score(df_to_cluster, kmeans_model_3.labels_)
print('Coeficiente de Silueta: {}'.format(sil_score_3))
# Evaluation (Inertia)
inertia_score_3 = kmeans_model_3.inertia_
print('Inercia: {}'.format(inertia_score_3))
# Evaluation (Davies-Bouldin Score
db_score_3 = davies_bouldin_score(df_to_cluster, kmeans_model_3.labels_)
print('Davies-Bouldin Score: {}'.format(db_score_3))

In [ ]:
#Comparacion de Metricas
dic_metric = {
                'N° Cluster':[3,4,10],
                'Silueta':[sil_score,sil_score_2,sil_score_3],
                'Inercia' : [inertia_score,inertia_score_2,inertia_score_3],
                'Davies-Boulding Score': [db_score,db_score_2,db_score_3]
                }
df_metric = pd.DataFrame(dic_metric)
df_metric

## Modelo con DBScan

In [ ]:
k = 6
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(df_to_cluster)
distances, indices = neighbors_fit.kneighbors(df_to_cluster)
distances = np.sort(distances[:, k-1], axis=0)

# Graficar
plt.figure(figsize=(8, 6))
plt.plot(distances)
plt.grid(True, which='both', linestyle='--', linewidth=0.7)

plt.xlabel('Puntos Ordenados')

plt.ylabel('Distancia k-ésima')
plt.yticks(np.arange(0, max(distances) , 0.1))

plt.show()

In [ ]:
#Vemos en la figura un epsilon de 0.5 como mejor referencia
# Ajustar modelo DBScan
dbs_model = DBSCAN(eps=.6, min_samples=6)
dbs_model.fit(df_to_cluster)

In [ ]:
# Recuento de inviduos por cada Clúster
resumen = pd.Series(dbs_model.labels_).value_counts()
resumen = pd.DataFrame(resumen)

resumen.index.name = 'Clúster'
resumen.columns = ['Cantidad']
resumen

In [ ]:
# Asignar el clúster a cada fila del Dataset original (sin escalar)
df_sin_outliers['cluster_DBScan'] = dbs_model.labels_
df_sin_outliers.head()

In [ ]:
# Plotear

# Scatter
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(df_sin_outliers['Year_Birth'], df_sin_outliers['Education'], df_sin_outliers['Income'],
                    c=df_sin_outliers['cluster_DBScan'],
                    cmap='viridis',
                    marker='o')


ax.set_xlabel('Year_Birth')
ax.set_ylabel('Education')
ax.set_zlabel('Income')
plt.colorbar(scatter, label='Cluster')

# Rotar el gráfico
ax.view_init(elev=20, azim=30)

plt.show()

In [ ]:
#Metricas de Evaluacion
# Evaluation (Silhouette Coefficient)
sil_score_dbs = silhouette_score(df_to_cluster, dbs_model.labels_)
print('Coeficiente de Silueta: {}'.format(sil_score_dbs))
# Evaluation (Davies-Bouldin Score
db_score_dbs = davies_bouldin_score(df_to_cluster, dbs_model.labels_)
print('Davies-Bouldin Score: {}'.format(db_score_dbs))

In [ ]:
#Visualizacion de los datos para el analisis
df_sin_outliers.head()

In [ ]:
#Visualizar los datos de cada cluster 

cluster_dbsca = {}
unique_cluster = df_sin_outliers['cluster_DBScan']
for i in unique_cluster:
    cluster_dbsca[i]=df_sin_outliers[df_sin_outliers['cluster_DBScan'] == i]
    
for i in range(12): 
    print(f'\nCluster N°{i+1}')
    print(cluster_dbsca[i][['Year_Birth','Education','Income']].describe())